In [1]:
!pip install pytesseract

In [2]:
!pip install selenium

In [3]:
import cv2
import pytesseract
from PIL import Image
import re
from selenium import webdriver
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [4]:
image = cv2.imread('Psych_prescription.jpeg')
img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#print(pytesseract.image_to_string(img))

blur = cv2.GaussianBlur(img,(5,5),0)
ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
size_structuring_element = 10
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (size_structuring_element, size_structuring_element))
dilation = cv2.dilate(th, rect_kernel, iterations = 1) 
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_LIST,  
                                                 cv2.CHAIN_APPROX_NONE)
meds = {}
  
# Looping through the identified contours 
# Then rectangular part is cropped and passed on 
# to pytesseract for extracting text from it 
# Extracted text is then written into the text file 
im2 = img.copy()
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
      
    # Drawing a rectangle on copied image 
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2) 
      
    # Cropping the text block for giving input to OCR 
    cropped = im2[y:y + h, x:x + w] 
      
    # Apply OCR on the cropped image 
    text = pytesseract.image_to_string(cropped)
    print('-------')
    print(text)

print(meds)

-------
 

 

Dr. Sasha Raikhy
MBBS, DNB - Psychiatry
Medical Registration Number: 72759

Positive Vibes, 82/20 Near Mahendra showroom, Safdarjung
Enclave, Delhi

Contact: https:/iwww_practo.com/consultidirect/chat-support

Tarun Sharma Prescription Id: 684462
24 yrs, Male ‘08/08/2020
Medicines

Prodep 20 Capsule
1—0—0
Dally after food, for 9 Days

Clonazepam (0.25 mg) Tablet MD
o—o—1
Dally after food, for 9 Days

 

7

Doctor's Signature
08/08/2020

epractoe

 

 

{}


In [5]:
text = pytesseract.image_to_string(img, lang='eng', config='--psm 11') 
is_next_med = 0

meds = {}
current_med = ''
for line in text.split('\n'): 
    if(line.strip().lower() == 'medicines'):
        is_next_med = 1
    elif(is_next_med == 1 and line.replace(' ','')!=''):
        meds[line] = []
        current_med = line
        is_next_med += 1
    elif(is_next_med == 2 and line.replace(' ','')!=''):
        found = re.search(".*[—].*[—].*", line)
        if found is None:
            # This means we have read all the meds already!
            meds.pop(current_med, None)
            break
        meds[current_med].append(line)
        is_next_med += 1
    elif(is_next_med == 3 and line.replace(' ','')!=''):
        meds[current_med].append(line)
        is_next_med = 1

print(meds)

{'Prodep 20 Capsule': ['1—0—0', 'Dally after food, for 9 Days'], '‘Clonazepam (0.25 mg) Tablet MD': ['o—o—1', 'Dally after food, for 9 Days']}


In [ ]:
# Aim: Add to my cart on 1mg
cap = DesiredCapabilities.CHROME
cap['chrome.switches'] = ['--disable-local-storage']
options = webdriver.ChromeOptions()
options.add_argument("--user-data-dir=/home/tarun/.config/google-chrome/Default")
options.add_argument("--disable-extensions")
options.add_argument("--incognito");
options.add_experimental_option('prefs', {
    'geolocation': False
})
options.add_argument("--set-headless")
options.add_argument("--enable-strict-powerful-feature-restrictions")
chrome_driver =os.getcwd() + '/chromedriver'
os.environ["webdriver.chrome.driver"] = chrome_driver
driver = webdriver.Chrome(executable_path=chrome_driver,options=options, desired_capabilities=cap)

# step1: login
driver.get('https://www.1mg.com/')
# Find anchor tag with child span's text == Login, click on it
wait = WebDriverWait(driver, 10)
wait.until(EC.visibility_of_element_located((By.XPATH,"//a[.//span[text()='Login']]")))
login_button = driver.find_element_by_xpath("//a[.//span[text()='Login']]")
login_button.click()
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.XPATH,"//label[text()='Enter Email ID or Mobile Number']")))
# Find label with text = 'Enter Email ID or Mobile Number', select its sibling div ka input tag, send keys
phone_input = driver.find_element_by_xpath("//label[text()='Enter Email ID or Mobile Number']/following-sibling::div/input")
phone_input.send_keys(7993267390)
# Find span with text 'SEND OTP', and click on its parent anchor

wait = WebDriverWait(driver, 10)
wait.until(EC.visibility_of_element_located((By.XPATH,"//a[//span[text()='SEND OTP']]")))

phone_submit = driver.find_element_by_xpath("//a[//span[text()='SEND OTP']]")
phone_submit.click()

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.XPATH,"//label[text()='One Time Password']")))
# Find label with text = 'One Time Password', select its sibling div ka input tag, ask user for otp input, send keys
otp_input = driver.find_element_by_xpath("//label[text()='One Time Password']/following-sibling::div/input")
# val = input('Enter OTP sent to your phone:')
# otp_input.send_keys(int(val))

# wait = WebDriverWait(driver, 10)
# wait.until(EC.element_to_be_clickable((By.XPATH,"//a[//span[text()='DONE']]")))

# otp_submit = driver.find_element_by_xpath("//a[//span[text()='DONE']]")
# otp_submit.click()
wait = WebDriverWait(driver, 30)
wait.until(EC.url_changes('https://www.1mg.com'))


# search 1mg/other delivery services for these meds
for med in meds:
    # step2: search for med
    driver.get(f'https://www.1mg.com/search/all?name={med}')
    wait = WebDriverWait(driver, 10)
    # Find first DIV with text ADD and click on that div
    first_item = driver.find_element_by_xpath("//div[text()='ADD']")
    first_item.click()
    
# Step3: then logout
driver.get('https://www.1mg.com/logout')
driver.close()